# A lecture on making code **fast**
(Designed for quantitative macro and similar structural econ in Python, but relevant for everyone!)

# First meta-principle: "premature optimization is the root of all evil"
From TeX inventor and computer scientist extraordinaire Donald Knuth:
> Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered. We should forget about small efficiencies, say about 97% of the time: **premature optimization is the root of all evil**.

# First meta-principle: "premature optimization is the root of all evil"
Actual full quote:
> Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered. We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil. **Yet we should not pass up our opportunities in that critical 3%.**

# Second meta-principle: profile, profile, profile

- How do you avoid premature optimization and identify that "critical 3%"?
- **Profiling** your code!
- Profiling tells you how much time (or memory) each part of your code takes to run
- Your tool *before* thinking about optimization

# What profiling looks like (line profiling)

After typing `%load_ext line_profiler` and `%lprun -f sim.backward_iteration sim.steady_state(**sim.example_calibration())` after importing the `sim_steady_state_fast.py` module at https://github.com/shade-econ/nber-workshop-2023/blob/main/Lectures/sim_steady_state_fast.py as `sim` , we get the results of *line profiling* the backward iteration function in our code, as it is applied to solve for the steady state. (I'm eliminating the comment lines for brevity.)

    Timer unit: 1e-09 s
    
    Total time: 0.021981 s
    File: /Users/matthewrognlie/Dropbox/joint folders/HANK minicourse/NBER workshop 2023/Lectures/sim_steady_state_fast.py
    Function: backward_iteration at line 89
    
    Line #      Hits         Time  Per Hit   % Time  Line Contents
    ==============================================================

    89                                               def backward_iteration(Va, Pi, a_grid, y, r, beta, eis):
    91          542    3848000.0   7099.6     17.5      Wa = beta * Pi @ Va
    94          542     981000.0   1810.0      4.5      c_endog = Wa**(-eis)
    95          542    3420000.0   6310.0     15.6      coh = y[:, np.newaxis] + (1+r)*a_grid
    97          542    9977000.0  18407.7     45.4      a = interpolate_monotonic_loop(coh, c_endog + a_grid, a_grid)
    101         542     363000.0    669.7      1.7      setmin(a, a_grid[0])
    102         542    1083000.0   1998.2      4.9      c = coh - a
    105         542    2208000.0   4073.8     10.0      Va = (1+r) * c**(-1/eis)
    107         542     101000.0    186.3      0.5      return Va, a, c

# What profiling looks like, continued

Typing `%prun sim.steady_state(**calib)` in the same notebook, having stored the calibration in `calib`, we get the results of Python's standard profiling of the steady-state function. `tottime` is the amount of time in that function specifically (not including other functions that it calls), while `cumtime` is the amount of time in the function total (including functions that it calls).

    4046 function calls in 0.038 seconds

    Ordered by: internal time
    
    ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        542   0.018    0.000    0.028    0.000 sim_steady_state_fast.py:89(backward_iteration)
        542   0.010    0.000    0.010    0.000 sim_steady_state_fast.py:168(interpolate_monotonic_loop)
        581   0.005    0.000    0.005    0.000 sim_steady_state.py:137(forward_policy)
        581   0.003    0.000    0.009    0.000 sim_steady_state.py:151(forward_iteration)
        1     0.001    0.001    0.029    0.029 sim_steady_state_fast.py:110(policy_ss)
        1128  0.000    0.000    0.000    0.000 serialize.py:30(_numba_unpickle)
        1     0.000    0.000    0.009    0.009 sim_steady_state_fast.py:237(distribution_ss)

# Very simple profiling

We can also use `%time` to figure out how long evaluating a single line takes (`%%time` if we want to time an entire Jupyter cell).

Subtlety: "Wall time" is literally how much time elapsed while running the command, "CPU times" is how much total time CPUs spent on the command specifically (wall time might be higher if computer did other stuff while running, CPU time might be higher if used multiple cores).

In [1]:
%time sum(i for i in range(10000))

CPU times: user 269 µs, sys: 15 µs, total: 284 µs
Wall time: 414 µs


49995000

In [2]:
%%time
x = 0
for i in range(10000):
    x += i

CPU times: user 519 µs, sys: 5 µs, total: 524 µs
Wall time: 584 µs


# Alternative

If we use `%timeit` rather than `%time`, the line is run many times rather than one, and various other measures are taken to get a more accurate, usually lower, measure of how long the code takes. 

In [3]:
%timeit sum(i for i in range(10000))

231 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
%%timeit
x = 0
for i in range(10000):
    x += i

201 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Summing up profiling
- Today we'll mainly use `%timeit` to profile simple fragments of code
- On more complicated projects, we need things like `%lprun` (line profiling) and `%prun` (profiling how long each function takes) to get a handle on things
- Commands starting with `%` are Jupyter/IPython "magic" commands, not Python code
    * but all these profiling tools can be invoked from regular Python code if you import the right module (e.g. the `timeit` module)
- Every language has profiling tools (though some are better or easier to use than others)

# Caveat before we get started
- Won't talk here about **parallelization** or **GPUs**, even though both can be important for the most intensive applications

- But many lessons here will carry over...
    * Profiling, keeping track of big-O complexity, reducing to efficient operations like matrix multiplication that are easily parallelized and put on GPUs...

- Computation times today will be from running notebook on my Macbook Air M2 laptop

# Now on to the main content...

# Six principles
1. Compiled is quick, the interpreter inches
2. Memory matters
3. Big O is a big deal
4. Favor fast functions
5. Savor sparsity and structure
6. Don't duplicate drudgery

# 1. Compiled is quick, the interpreter inches
- Some languages (e.g. C, C++, Fortran) require *compilation* to machine code before running, which is then executed directly by the processor (fast)
- Other languages (e.g. Python, R, Matlab) are *interpreted*: an "interpreter" pretty much directly runs your commands
    - Takes a lot longer to run, because the interpreter has to do a lot of housekeeping (e.g. look up where a variable is stored, what type it is, what a command means for that type)
    - But it's more interactive and easier to get started, since there's no annoying compilation step
    - Also makes it easy to avoid doing housekeeping yourself like declaring variables

# Example: add two random arrays, loop vs. direct
300x-400x speed difference?!

In [5]:
import numpy as np
x = np.random.rand(1000)
y = np.random.rand(1000)

In [6]:
%%timeit
z = np.empty(1000)
for i in range(1000):
    z[i] = x[i] + y[i]

148 µs ± 4.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%timeit z = x + y

433 ns ± 10.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# What's going on?
- When we use a pure Python construct like a loop, it uses the interpreter
- Other things are function calls passed through interpreter too
    - Look up element `i` in an array, add two numbers, assign element `i` in an array
- When we write `z = x + y`, the NumPy package calls compiled code under the hood to do the addition fast
- These "vectorized" commands avoid spending much time in the interpreter, because we're giving high-level instructions to compiled code
- Most of you probably think of this as rule 1: vectorized good, loops bad

# Do loops need to be bad?
An amazing package called **Numba** allows us to do "just-in-time" compilation of many Python functions (with NumPy numerical code) just by applying a "decorator" to the function.

Let's apply this to the inefficient loop code we wrote.

In [8]:
import numba

@numba.njit
def add(x, y):
    z = np.empty(1000)
    for i in range(1000):
        z[i] = x[i] + y[i]
    return z

# Numba results
The first time we run it, it's slow, because Numba has to compile the function and that takes time.

In [9]:
%time z = add(x, y)

CPU times: user 188 ms, sys: 34.4 ms, total: 223 ms
Wall time: 300 ms


But after that, it has a speed not far from directly writing `z = x + y`.

In [10]:
%timeit z = add(x, y)

636 ns ± 16.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# How slow is interpreted code in general?
We saw almost a 400x speed difference in one case. It's useful to experiment with different examples to get a sense of relative cost.

Let's compare code that adds these 1000-element vectors to code that retrieves items from a dictionary that maps 1000 numbers to their squares. This is more like a 60x difference: here, addition of NumPy vectors takes slightly less than 0.5 ns per element, while loop with integer dictionary accesses takes just above 25 ns per element (still quite fast).

In [11]:
%timeit z = x + y

444 ns ± 4.36 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
numbers_to_squares = {i: i**2 for i in range(1000)}

In [13]:
%%timeit
for i in range(1000):
    square = numbers_to_squares[i]

27.1 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Be careful: use NumPy functions to operate on arrays
- Common way to trip up beginners!
- There's a standard Python function called `sum` that can do a sum over anything that's iterable using a for loop - basically, it runs a for loop for you.
- But this spends a lot of time in the interpreter, so is slow, and you want the NumPy function `np.sum`: another case of the interpreter inching along, and compiled being quick

In [14]:
%timeit sum(x)

38.3 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
%timeit np.sum(x)

1.44 µs ± 23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# 2. Memory matters
- Here's where we start to get into things you *don't* know!
- It turns out that reads and writes to *memory*, not processing by the CPU itself, is often the speed bottleneck
- Huge speed differences in code possible when we avoid needless reads and writes, and keep as much as possible in the processor's *cache*
- A lot of people who think, say, that Fortran is inevitably way faster than Python, don't understand this

# What to do when memory matters? Avoid temporaries!
Suppose we want to evaluate $z = f(x,y) \equiv x^2 + 3xy + y^2 - 2x - 4y +1$.

Naive (though easy-to-read) option is to just write it out:

In [16]:
%timeit z = x**2 + 3*x*y + y**2 - 2*x - 4*y + 1

6.53 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# From Python's perspective, what's happening is similar to...
Writing and reading a ton of intermediate arrays: a lot of wasteful dealing with memory!

In [17]:
%%timeit
z1 = x**2
z2 = x*y
z3 = 3*z2
z4 = y**2
z5 = 2*x
z6 = 4*y
z7 = 1
z = z1 + z3 + z4 - z5 - z6 + 1

6.64 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Compiled with Numba, a loop would be much faster
Because they directly do the computation with $x$ and $y$ one entry at a time, with everything probably staying in the processor's registers, and run through the arrays in order without creating intermediates (which helps computer efficiently load memory into cache one block at a time), compiled loops are actually *much better than vectorization* - 7 times faster in this case!

In [18]:
@numba.njit
def f(x, y):
    z = np.empty_like(x)
    for i in range(len(x)):
        z[i] = x[i]**2 + 3*x[i]*y[i] + y[i]**2 - 2*x[i] - 4*y[i] + 1
    return z

In [19]:
f(x, y) # run once to compile
%timeit f(x, y)

899 ns ± 14.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# That's right, (compiled) loops can be *better* than vectorization!

# But sometimes loops are uglier...
- One problem with this strategy is that it's often very nice to be able to just write an expression directly in vectorized form
- Loops add unnecessary housekeeping and make the code less clean
- Certainly don't do this if it's not really important to speed up your code (avoid premature optimization)
    * Also, tools like JAX work are designed for vectorized rather than loop-based code

# Cleaner way
Fortunately, Numba provides a cleaner way to do this, the "vectorize" decorator, which turns a function on scalars into a compiled function that will evaluate element-by-element on vectors (or larger arrays).

Instead of making your code less clean, this can actually make your code *more* clean - since it's good practice to put fairly-complex calculations into their own functions and give them descriptive names (more descriptive than `f`).

In [20]:
@numba.vectorize
def f(x, y):
    return x**2 + 3*x*y + y**2 - 2*x - 4*y + 1

In [21]:
f(x, y) # run once to compile
%timeit f(x, y)

1.1 µs ± 14.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Caveat: other languages
- This gives a huge improvement in Python because in Python, all those unnecessary temporary arrays are always created, at great cost, unless you specify otherwise
- Matlab sometimes is smart and avoids this
    - but not always, and you never know when
- If the compiler is smart enough in a compiled language with arrays (e.g. Fortran), it will avoid this
    - but again very dependent on the language, compiler, etc., not super predictable
    - get the most predictable performance by writing loops
- In Julia, you can avoid this by using its "dot" syntax (basically its analog of `numba.vectorize`)

# More memory matters: taking dot product
Often need to do this when aggregating individual variables across some distribution.

First version creates a temporary `x*y` array and then sums: much slower!

In [22]:
%timeit np.sum(x*y)

1.89 µs ± 2.82 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [23]:
%timeit np.vdot(x, y)

808 ns ± 5.66 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Even more memory mattering...
Two identical computations, but wasteful to create a vector filled with ones and then have it be accessed to do addition, rather than just putting a 1 there and letting NumPy handle it.

In [24]:
%timeit np.ones(1000) + x + y

1.65 µs ± 8.97 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [25]:
%timeit 1 + x + y

1.24 µs ± 4.89 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# 3. Big O is a big deal
- Now we're onto some (very basic) computer science theory
- Suppose the size of a problem is in some way described by $N$ (e.g. we're operating on an array with $N$ entries)
- If we say that the time for an algorithm is $O(f(N))$, that means that for large enough $N$, the cost of the algorithm is bounded from above by $M\cdot f(N)$, where $M$ is some constant
- Good for getting a handle on cost: $O(\log N)$ is log, $O(N)$ is linear, $O(N \log N)$ is log times linear, $O(N^2)$ is quadratic, etc.
- For simple algorithms can get it by counting operations (or otherwise informally)
- Called **Big O notation** if you want to read further

# Problem: locating elements in array
- Problem: for each element of `X`, find which elements of `Y` it's between
- Suppose `X` and `Y` are sorted vectors, and we want to return a vector that says for each element `X[i]`, the index `j` such that `X[i]` is between `Y[j]` and `Y[j+1]`
- If `X[i]` is lower than all elements of `Y`, put -1, and if it's greater than all elemnts of `Y`, put -2

# Simple $O(N^2)$ code to handle this
Loop through: for each element `X[i]`, go through all the elements `Y[j]` until we find first one that's bigger, then we know it's between `Y[j-1]` and `Y[j]`.

In [26]:
@numba.njit
def find_indices(X, Y):
    js = np.full(len(X), -2) # default -2 if we never find it
    for i in range(len(X)):
        for j in range(len(Y)):
            if X[i] < Y[j]:
                js[i] = j-1
                break
    return js

In [27]:
find_indices(np.array([1, 2, 2.8, 3]),
             np.array([1.5, 2.5, 2.7, 2.9]))

array([-1,  0,  2, -2])

# Can we see the $O(N^2)$?
Supposing that `X` and `Y` are both length $N$, this will be $O(N^2)$, since we have to loop through all elements of $X$ (that's one factor of $N$), and then on average if we have to loop through half the elements of `Y` to find one we have $N/2$ (another factor of $N$ when we drop constants).

When we increase the size of the problem by a factor of 10, time taken grows by a factor of 100:

In [28]:
X = np.sort(np.random.rand(1000))
Y = np.sort(np.random.rand(1000))
%timeit find_indices(X, Y)

153 µs ± 2.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
X = np.sort(np.random.rand(10000))
Y = np.sort(np.random.rand(10000))
%timeit find_indices(X, Y)

14.9 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Alternative algorithm: $O(N \log N)$ binary search
Since we know `Y` is sorted, there's a better way to search than just running linearly through all elements of `Y`. We can do a *divide and conquer* approach, called a binary search (bisection is the continuous version). Let's code this up to find any `x` in `Y`, then write as a loop.

In [30]:
@numba.njit
def binary_search(x, Y):
    if x < Y[0]:
        return -1
    elif x > Y[-1]:
        return -2
    else:
        low = 0
        high = len(Y) - 1
        while high - low > 1:
            mid = (low + high) // 2 # // operator is "division with round down to nearest integer"
            if x < Y[mid]:
                high = mid
            else:
                low = mid
        return low
    
@numba.njit
def find_indices_binary(X, Y):
    js = np.empty(len(Y), dtype=np.int64)
    for i in range(len(X)):
        js[i] = binary_search(X[i], Y)
    return js

In [31]:
find_indices_binary(np.array([1, 2, 2.8, 3]),
                    np.array([1.5, 2.5, 2.7, 2.9]))

array([-1,  0,  2, -2])

# Can we see the $O(N \log N)$?
It turns out that each application of the binary search takes $O(\log N)$, because we halve the size of the interval each time, so the number of steps is approximately $\log_2(N)$, and then we drop the constant for big O. So doing this for every element of `X` is $O(N \log N)$.

Sure enough, we increase at a rate slightly worse than linear when we raise the size of the problem by 10. Also, the first case is almost 30 times faster than before.

In [32]:
X = np.sort(np.random.rand(10000))
Y = np.sort(np.random.rand(10000))
%timeit find_indices_binary(X, Y)

485 µs ± 5.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [33]:
X = np.sort(np.random.rand(100000))
Y = np.sort(np.random.rand(100000))
%timeit find_indices_binary(X, Y)

6.81 ms ± 58.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Best algorithm: $O(N)$ linear search exploiting `X` sorted
The previous algorithm used the fact that `Y` was sorted to do a binary search (divide and conquer) and speed things up from $O(N^2)$ to $O(N \log N)$. No assumptions were needed for `X`.

But we actually assumed that `X` was sorted too. That means we don't have to start from the beginning of `Y` each time: we know that the index of `X[i+1]` will be higher than that of `X[i]`.

Just need to make *slight* modification to our first function.

In [34]:
@numba.njit
def find_indices_smart(X, Y):
    js = np.full(len(X), -2) # default -2 if we never find it
    j = 0
    for i in range(len(X)):
        for j in range(j, len(Y)): # start from where you ended up last time!
            if X[i] < Y[j]:
                js[i] = j
                break
    return js

In [35]:
find_indices_smart(np.array([1, 2, 2.8, 3]),
                   np.array([1.5, 2.5, 2.7, 2.9]))

array([ 0,  1,  3, -2])

# Can we see the $O(N)$?
Once we've determined that `X[i]` is between `Y[3]` and `Y[4]`, we start our search for `X[i+1]` between `Y[3]` and `Y[4]`, skipping earlier entries. This strategy avoids redundantly going through large parts of `Y` and turns out to actually have *linear* time $O(N)$.

The middle case is about 7.5 times faster than the comparable case for our $O(N \log N)$ algorithm, probably partly because it has a more regular memory access pattern too. (Note: growth still a bit more than linear in $N$ in practice, probably due to memory issues.)

In [36]:
X = np.sort(np.random.rand(10000))
Y = np.sort(np.random.rand(10000))
%timeit find_indices_smart(X, Y)

55.5 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [37]:
X = np.sort(np.random.rand(100000))
Y = np.sort(np.random.rand(100000))
%timeit find_indices_smart(X, Y)

887 µs ± 7.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [38]:
X = np.sort(np.random.rand(1000000))
Y = np.sort(np.random.rand(1000000))
%timeit find_indices_smart(X, Y)

9.58 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Bottom line
- Different algorithms to solve the same problem make a huge difference
- By learning to figure out the big-O complexity of an algorithm, you can quickly gauge
    - whether it might underperform on larger problems
    - whether it might be possible to improve (can we reduce $O(N^2)$ to $O(N \log N)$ to $O(N)$ like here?)
    
    
- Applies to all discrete operations
    - for continuous things where we need to converge to a solution within some tolerance (e.g. root finding), there are related other metrics

# 4. Favor fast functions
- Some functions are much faster than others, and knowing which can save a lot of time
    - under the hood, this might be for "memory matters" or "big O is a big deal" reasons

    
- For simple, core functions like sine or matrix multiplication, the built-in implementation will be usually much faster than what you can do
    - with a few notable exceptions...
    
    
- For more complex functions, it really depends

# Example: matrix multiplication
We could code this "by hand" with loops in Numba.

In [39]:
@numba.njit
def multiply_by_hand(X, Y):
    Z = np.zeros((X.shape[0], Y.shape[1]))
    for i in range(X.shape[0]):
        for j in range(Y.shape[1]):
            for k in range(X.shape[1]):
                Z[i, j] += X[i, k] * Y[k, j]
    return Z

Test to make sure we remembered matrix multiplication correctly:

In [40]:
X = np.random.rand(200, 200)
Y = np.random.rand(200, 200)
np.allclose(X @ Y, multiply_by_hand(X, Y))

True

# But we are a LOT slower than standard matrix multiplication
By a factor of over 20!

It turns out that matrix multiplication is *so* important that it's very, very efficiently implemented in most numerical libraries.

One reason for the greater speed on my machine is that it knows how to use multiple cores efficiently (which we didn't enable in our Numba code).

A million other subtle, careful things that experts know how to get right.

In [41]:
%timeit X @ Y

271 µs ± 9.64 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [42]:
%timeit multiply_by_hand(X, Y)

5.75 ms ± 50.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Example going the other way: powers
Turns out that powers (other than special cases 0, 1, -1, and 2) are extremely slow, especially in the implementation used by NumPy, compared to basic arithmetic operations like multiplication, by a factor of 10 or more. If everything else is optimized, this often ends up being a bottleneck in your code!

In [43]:
x = np.random.rand(1000) + 1

In [44]:
%timeit 2.3*x

615 ns ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [45]:
%timeit x**2.3

8.15 µs ± 45.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


So much that we can actually speed things up a bit if we actually do the power ourselves with exp and log.

In [46]:
%timeit np.exp(2.3*np.log(x))

6.04 µs ± 7.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Verify that the error is tiny:

In [47]:
relerr = np.exp(2.3*np.log(x)) / x**2.3 - 1
np.max(np.abs(relerr))

2.220446049250313e-16

This is [machine epsilon](https://en.wikipedia.org/wiki/Machine_epsilon), the minimum relative difference between two numbers in the computer's double-precision [floating point arithmetic](https://en.wikipedia.org/wiki/Floating-point_arithmetic):

In [48]:
np.finfo(float).eps

2.220446049250313e-16

So this is pretty much as accurate as we can possibly hope, short of exact accuracy up to machine precision.

Why is the computer's power function actually slower than this (and why doesn't it just use the `exp` `log` trick instead)? It's hard to say: could be obscure implementation details, like spending some time optimizing special cases. Or it could be the fact that it's costly to make sure that the number is *exactly* right up to rounding to machine precision, which can be a lot harder than having a machine epsilon error like we did above.

# Polynomial evaluation
Suppose we want to evaluate a polynomial like $6x^4 + 3x^3 + 2x^2 - 4x + 4$ for all points in a large vector `x`.

In [49]:
x = np.random.rand(10000)
p = np.array([6, 3, 2, -4, 4])

Method 1: direct evaluation

In [50]:
%timeit p[0]*x**4 + p[1]*x**3 + p[2]*x**2 + p[3]*x + p[4]

183 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Method 2: NumPy's function

In [51]:
%timeit np.polyval(p, x)

40.4 µs ± 2.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Why is NumPy doing 4 to 5 times better than ours? Well, we already have two ideas:
- First, non-special-case powers are slow, so `x**4` and `x**3` are inefficient.
- Second, for "memory matters" reasons, writing out vectorized expressions explicitly like this is slow.

# Faster polynomial evaluation
Method 3: use multiplication rather than powers. This already gets almost as fast as NumPy, a 5-fold improvement on the original!

In [52]:
%timeit p[0]*x*x*x*x + p[1]*x*x*x + p[2]*x*x + p[3]*x + p[4]

50 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Method 4: [Horner's method](https://en.wikipedia.org/wiki/Horner%27s_method), the standard efficient way to evaluate polynomials. This rearranges the expression above in a smart way to reduce the number of multiplication signs `*` from 10 to 4, while keeping the number of addition signs `+` the same.

In [53]:
%timeit p[4] + x*(p[3] + x*(p[2] + x*(p[1] + p[0]*x)))

22.5 µs ± 536 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


It turns out that this is what NumPy's `polyval` function uses, but by avoiding the overhead of the function we're already doing better than it (and almost 10 times better than our original expression)!

# Fasterrrrrrrrrrrr!!!!!!!!
We know from "memory matters" that it's inefficient to have so many temporary arrays created, so let's use `numba.vectorize`. This speeds us up by a factor of almost 10, so that we are 50 times faster than our original code!

In [54]:
@numba.vectorize
def f_horner(x):
    return p[4] + x*(p[3] + x*(p[2] + x*(p[1] + p[0]*x)))

In [55]:
f_horner(x)
%timeit f_horner(x)

3.83 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# You would have thought that writing the polynomial out directly, in vectorized code, was a reasonable approach... but here we just did 50 times better!

# Bottom line
- This was a bit of a grab-bag...


- But the main idea is that there's quite a bit of variation in the speeds that different functions take, even when you think it's obvious or that it can't be that different:
    - NumPy did matrix multiplication 20 times faster than us...
    - but with Numba we evaluated a polynomial 10 times faster than NumPy's `polyval`!
    - powers are really slow, though we can make them a bit faster
    
    
- A few simple rules can get you most of the way
    - most elementary functions and also key mathematical operations like matrix multiplication and the FFT are implemented super-fast
    - other stuff more questionable
    
- But there's no substitute for using a lot of `%timeit` if you want to become a guru of these things

# 5. Savor sparsity and structure
I just told you that matrix multiplication is really, really fast.

But one foolish thing a lot of people do is to transcribe their matrix notation directly into code, without thinking about what's going on.

If you don't take advantage of *sparsity* and *structure*, you can end up with extremely costly large matrix operations - since multiplying or inverting $N$-by-$N$ matrices is cubic: $O(N^3)$.

# Independent laws of motion
It's fairly common to have two states that obey independent laws of motion. We often simplify notation by combining these two states into one state, and combining the Markov transition matrices using the Kronecker product.

Suppose these states can each have 50 values apiece. Let `D` by a 50-by-50 distribution across two states, and let them have independent laws of motion `Pi1` and `Pi2`.

In [56]:
D = np.random.rand(50,50)
Pi1 = np.random.rand(50,50)
Pi2 = np.random.rand(50,50)

Use the Kronecker product to get the combined 2500-by-2500 transition matrix `Pi`. (Note that this is *huge*, with $2500^2 / (2\cdot 50^2) = 1250$ times more entries than `Pi1` and `Pi2` combined.)

In [57]:
Pi = np.kron(Pi1, Pi2)

# What's faster, independent updates or the Kronecker product?

Kronecker product approach: apply the Markov transition matrix `Pi` (on the right) to the flattened distribution (we use `D.ravel()` to flatten - unlike `D.flatten()`, it doesn't make a copy):

In [58]:
%timeit D.ravel() @ Pi

2.29 ms ± 344 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Other approach: apply the Markov transition matrices `Pi1` and `Pi2` separately to the two-dimensional array `D`.

In [59]:
%timeit Pi1.T @ D @ Pi2

14.4 µs ± 210 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Difference in speed is over a factor of 100! What's responsible for this? In principle, the first approach is one 2500-by-2500 times 2500 matrix-vector multiplication, complexity $2500^2$, vs. the second approach which is two 50-by-50 times 50-by-50 matrix-matrix multiplications, complexity $2\times 50^3$.

The ratio of these two is only 25 - how did we get an even bigger difference in speed? Probably because *memory matters*: much more can stay in the cache and be accessed quickly if we're only dealing with a few 50-by-50 matrices than if we're dealing with a 2500-by-2500 matrix.

Finally, verify that the results are the same:

In [60]:
np.allclose(D.ravel() @ Pi, (Pi1.T @ D @ Pi2).ravel())

True

# Structure: exploit it!
Moral of the story: turns out that writing everything in standard matrix-vector notation, even if there's a more concise way to represent objects, is a really bad idea in computation.

# Trick to help out: einsum
One trick for keeping track of coordinates in messy multidimensional problems is to use the brilliant `np.einsum` command - a bit slower than matrix multiplication (sometimes much slower if it's not "optimized") but extremely transparent once you get the hang of it, infinitely better than the traditional Matlab cavalcade of transposes and swapping axes and Kronecker products.

The following says that we start with a distribution with coordinates $i,j$, and `Pi1` takes coordinate $i$ to $k$ while `Pi2` takes coordinate $j$ to $l$, to get a new distribution with coordinates $k,l$. (In this case, the `->kl` actually isn't necessary, but I have it there to be explicit. This "Einstein" notation just says to take a sum over dimensions $i$ and $j$ of all terms $D_{i,j}\Pi^1_{i,k}\Pi^2_{j,l}$.)

In [61]:
result = np.einsum('ij,ik,jl->kl', D, Pi1, Pi2)
np.allclose(result, Pi1.T @ D @ Pi2)

True

**Side note**: This is actually a (rare) case where the default `einsum` is much slower. We need to feed `optimize=True` for it to go fast (or precalculate the optimal path using `einsum_path` and write it into the code), but that fixed effort of doing so takes too many microseconds for ultra-tiny problems like this one.

For larger examples, there can still be a performance hit relative to matrix multiplication, but not always:

In [62]:
D = np.random.rand(500,500)
Pi1 = np.random.rand(500,500)
Pi2 = np.random.rand(500,500)

In [63]:
%timeit Pi1.T @ D @ Pi2

6.71 ms ± 418 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
%timeit np.einsum('ij,ik,jl->kl', D, Pi1, Pi2, optimize=True)

6.19 ms ± 78.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Sparse transition rules
Another very common occurence is that we have *sparse* Markov transition matrices that describe movements between states.

For instance, suppose that we have 1000 states (perhaps these are gridpoints for assets), and the agent in state $i$ has a policy of going to state $i+1$ with probability $1/(i+1)$, and staying at state $i$ with probability $i/(i+1)$ - except at the top state, where she stays with probability 1.

How do we apply this Markov matrix to update a distribution? A **very bad** approach is to explicitly construct the Markov matrix `Pi`.

In [65]:
Pi = np.diag(np.arange(1000) / (np.arange(1000)+1)) + np.diag(1 / (np.arange(999)+1), 1)
Pi[-1, -1] = 1

In [66]:
D = np.random.rand(1000)
%timeit D @ Pi

376 µs ± 50.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Built-in matrix multiplication is so fast that this isn't *horrible*, but still it's incredibly inefficient: we're using a 1000-by-1000 matrix even though almost all its entries are zeroes!

# Better approaches
One less-terrible approach, which is very popular, is to still explicitly construct `Pi`, but do so with sparse matrix libraries. This has some advantages - we get to stick to matrix notation - but it can still be annoying to deal with sparse matrix libraries, and often slower because we can't specialize to our specific case.

The best approach is simply to *use loops* - compiled with Numba, of course - to directly implement this. Often we can be a bit clever in writing the function to minimize memory accesses or computation:

In [67]:
@numba.njit
def update(D):
    Dnew = D.copy()
    for i in range(len(D)-1):
        movers = D[i] / (i+1)
        Dnew[i] -= movers
        Dnew[i+1] += movers
    return Dnew

Result: almost 400 times faster than the naive matrix multiplication!

In [68]:
update(D)
%timeit update(D)

1.04 µs ± 0.827 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Real life: these issues are often combined
- It's not an accident that I gave these examples!
- In het-agent models, it's quite common that the Markov matrix combines several independent processes (or at least smaller Markov matrices that can be applied sequentially), and some of these are sparse.
- Extreme efficiency or inefficiency possible depending on whether we adopt the lessons here:
    - Apply separate Markov matrices rather than using Kronecker products
    - Directly write code to handle sparse transitions rather than building matrices
- Of course, these lessons are more important when the number of states is large and the $O(N^3)$ of ordinary matrix multiplication really bites.

# 6. Don't duplicate drudgery

- Often the same calculation is needed in different places in your code


- Sounds obvious, but you don't want to repeat this calculation if at all possible!
    * Harder than it sounds!
    
    
- Figuring out what expensive things you can "precompute" and then reuse many, many times is an important trick
    * Tricky real-life examples will have to wait until we're looking at more complex code, though

# Calculations in the right order
It's fairly common to need to multiply an array by a bunch of constants.

You should multiply all the constants together *first* and then multiply by the array, not vice versa, so that there's only one scalar-array multiplication, rather than a bunch of temporary arrays created. (This is also a "memory matters" issue.)

In [69]:
a = 2.3
b = 6.1
c = 3.7
d = -0.3
x = np.random.rand(1000)

Slow (calculates array `x*a`, then multiplies this by `b` to make a new array, etc.):

In [70]:
%timeit x*a*b*c*d

2.45 µs ± 3.23 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Fast (multiplies all the scalars together first):

In [71]:
%timeit (a*b*c*d)*x

655 ns ± 3.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


We actually don't need the parentheses, since evaluation is left-to-right:

In [72]:
%timeit a*b*c*d*x

657 ns ± 3.69 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Think if you've already solved a nearby problem
Suppose we're trying to calculate the $N$ complex roots of unity $e^{2 i \pi n/N}$ for $n=0,\ldots,N-1$, which go around the unit circle in the complex plane.

Naive approach (but at least multiplying scalars first, since we just learned that):

In [73]:
N = 10000
%timeit np.exp(2j*np.pi/N*np.arange(N))

111 µs ± 635 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Smarter approach: consecutive entries only differ by constant factor `np.exp(2j*np.pi/N)`, so we can calculate this *once* and then recursively build up.

In [74]:
@numba.njit
def smart_unity(N):
    roots = np.empty(N, dtype=np.complex128)
    roots[0] = 1
    factor = np.exp(2j*np.pi/N)
    for i in range(1, N):
        roots[i] = factor*roots[i-1]
    return roots

In [75]:
smart_unity(N)
%timeit smart_unity(N)

23.5 µs ± 20.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


We just saved a factor of 10! (And we could save some more if we realized that after we calculate half, the other next half is just the complex conjugates.)

Side note: interestingly, we could get these from the Fast Fourier Transform, which is very efficiently implemented on the computer, but because this is $O(N log N)$ rather than $O(N)$, it's actually a bit slower here:

In [76]:
assert np.allclose(np.fft.fft(np.arange(N)==(N-1)), smart_unity(N))
%timeit np.fft.fft(np.arange(N)==(N-1))

85.9 µs ± 2.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Don't unintentionally call the same function many times
Sometimes you call the same function many times because it's difficult to keep track of the hierarchy of functions, and where it might be called.

As an extreme (maybe contrived) example, imagine that we want to calculate the Fibonacci sequence and decide to do the recursive algorithm.

In [77]:
@numba.njit
def fib(n):
    if n < 2:
        # base cases fib(0)=0, fib(1)=1
        return n
    else:
        return fib(n-1) + fib(n-2)

In [78]:
fib(20)
%timeit fib(20)
%timeit fib(30)

27.6 µs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
3.4 ms ± 20.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The cost explodes exponentially because when we calculate `fib(30)`, we're for instance calling `fib(28)` twice (once directly and once for `fib(29)`), and `fib(5)` a zillion times.

If we write it iteratively instead, we do far better, because we only calculate each value once.

In [79]:
@numba.njit
def fib_iterative(n):
    if n < 2:
        return n
    
    fs = np.empty(n+1)
    fs[0] = 0
    fs[1] = 1
    
    for i in range(2, n+1):
        fs[i] = fs[i-1] + fs[i-2]
   
    return fs[n]

In [80]:
fib_iterative(20)
%timeit fib_iterative(20)
%timeit fib_iterative(30)
%timeit fib_iterative(10000)

175 ns ± 1.39 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
217 ns ± 1.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
36.8 µs ± 858 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# Refresher: memory matters, let's avoid reads and writes
Instead of previous code, which built up Fibonacci numbers in an array (which would, of course, be useful if we wanted that array for later), write the following that avoids memory reads and writes alogether:

In [81]:
@numba.njit
def fib_nomemory(n):
    if n < 2:
        return n
    
    f_2 = 0
    f_1 = 1
    
    for i in range(2, n+1):
        f_0 = f_2 + f_1
        f_2 = f_1
        f_1 = f_0
   
    return f_0

In [82]:
fib_nomemory(10000)
%timeit fib_iterative(10000)
%timeit fib_nomemory(10000)

35.6 µs ± 273 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
3.08 µs ± 87.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# All done
1. Compiled is quick, the interpreter inches
2. Memory matters
3. Big O is a big deal
4. Favor fast functions
5. Savor sparsity and structure
6. Don't duplicate drudgery

# Two closing thoughts

# A. Don't optimize prematurely!
- When you first write code, write it as simply and cleanly and intelligibly as you can
    - Maybe this code won't work on the real problem you have in mind, but it will probably work on a smaller version of the problem (less data, grids with fewer points, processes with fewer exogenous states, etc.)


- You can use this to explore an idea, and get test benchmarks where we're pretty sure we have the right answer (because the code is so transparent)


- *Then* as you profile and learn about bottlenecks, start to bring out all the tricks and get speedy

# B. Computational power isn't limited to your laptop
- For this lecture I'm running code on my personal machine, since it's the very simplest to set up, and with good coding practices we can do even quite complex problems


- These 10x or 100x or 1000x speedups are often useful no matter what setup you're using!


- But... cloud computing has made it a lot more convenient to call upon computing power as a commodity, so if there's some bottleneck, we can complement the tricks here by throwing cheap resources at the problem too
    * Also makes it easier to use GPUs, etc.